In [1]:
!pwd

/home/jovyan


In [3]:
!hdfs dfs -ls /data/wiki/en_articles_part

Found 1 items
-rwxrwxrwx   1 jovyan supergroup   76861985 2018-08-01 17:15 /data/wiki/en_articles_part/articles-part


In [11]:
!hdfs dfs -cat /datasets/stop_words_en.txt

a
about
above
across
after
afterwards
again
against
all
almost
alone
along
already
also
although
always
am
among
amongst
amoungst
amount
an
and
another
any
anyhow
anyone
anything
anyway
anywhere
are
around
as
at
back
be
became
because
become
becomes
becoming
been
before
beforehand
behind
being
below
beside
besides
between
beyond
bill
both
bottom
but
by
call
can
cannot
cant
co
computer
con
could
couldnt
cry
de
describe
detail
do
done
down
due
during
each
eg
eight
either
eleven
else
elsewhere
empty
enough
etc
even
ever
every
everyone
everything
everywhere
except
few
fifteen
fify
fill
find
fire
first
five
for
former
formerly
forty
found
four
from
front
full
further
get
give
go
had
has
hasnt
have
he
hence
her
here
hereafter
hereby
herein
hereupon
hers
herse"
him
himse"
his
how
however
hundred
i
ie
if
in
inc
indeed
interest
into
is
it
its
itse"
keep
last
latter
latterly
least
less
ltd
made
many
may
me
meanwhile
might
mill
mine
more
moreover
most
mostly
move
much
must
my
myse"
name
namely
ne

In [13]:
%%writefile mapper.py
#!/user/bin/env python
import sys
from line in sys.stdin:
    words = line.split()
for word in words:
    print('{0}\t{1}'.format(word,1))

Overwriting mapper.py


In [14]:
%%writefile reducer.py
#!/user/bin/env python
import sys

curr_word = None
curr_count = 0

for line in sys.stdin:
    word,count = line.split('\t')
    count = int(count)
    if word == curr_word:
        curr_count += count
    else:
        if curr_word:
            print('{0}\t{1}'.format(curr_word),curr_count)
        curr_word = word
        curr_count = count
    if curr_word == word:
        print('{0}\t{1}'.format(curr_word),curr_count)


Writing reducer.py


In [18]:
%%writefile wordcount.txt

hello how are you ?
where are you ?
what are you doing for a living ?
what experience are you packing up in this lifetime ?

Overwriting wordcount.txt


In [2]:
!ls

checkWordCount.ipynb  StopWordsTask2.ipynb  wordcount.txt
mapper.py	      supervisord.log	    WordsRatingTask1.ipynb
README.md	      supervisord.pid
reducer.py	      WordCountTask0.ipynb


In [9]:
! hdfs dfs -rmr /word

rmr: DEPRECATED: Please use '-rm -r' instead.
Deleted /word


In [26]:
! hdfs dfs -mkdir word

mkdir: `word': File exists


In [12]:
!hdfs dfs -ls word

In [13]:
!hdfs dfs -mv wordcount.txt word/

In [27]:
!hdfs dfs -cat word/wordcount.txt


hello how are you ?
where are you ?
what are you doing for a living ?
what experience are you packing up in this lifetime ?

In [28]:
%%bash

OUT_DIR="wordcount1_"$(date +"%s%6N")
NUM_REDUCERS=1

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input word/ \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000  | head


19/03/07 13:12:25 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/07 13:12:25 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/03/07 13:12:27 INFO mapred.FileInputFormat: Total input files to process : 1
19/03/07 13:12:27 WARN hdfs.DataStreamer: Caught exception
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1252)
	at java.lang.Thread.join(Thread.java:1326)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:927)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:578)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:755)
19/03/07 13:12:27 INFO mapreduce.JobSubmitter: number of splits:2
19/03/07 13:12:27 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1551938777455_0009
19/03/07 13:12:28 INFO impl.YarnClientImpl: Submitted application application_1551938777455_0009
19/03/07 13:12:28 INFO mapreduce.Job: The url to tra

In [ ]:
h